In [ ]:
target_emotions = ["sadness", "grief", "fear", "remorse", "disappointment", "nervousness", "embarrassment" ]  # custom order: high severity first

In [ ]:
# Load dataset
from datasets import load_dataset

goemo_dataset = load_dataset("go_emotions")

# Get label names (maps index to emotion string)
id2label = goemo_dataset['train'].features['labels'].feature.names
label2id = {label: i for i, label in enumerate(id2label)}

# Create emotion index map for only target emotions
target_label2id = {emotion: idx for idx, emotion in enumerate(target_emotions)}
target_id2label = {idx: emotion for emotion, idx in target_label2id.items()}
target_label_ids = {label2id[e] for e in target_emotions}

In [ ]:
import pandas as pd

# Convert to DataFrame
df = goemo_dataset["train"].to_pandas()

# Keep only rows with at least one target emotion
def get_first_target_label(labels):
    for l in labels:
        if l in target_label_ids:
            return id2label[l]
    return None

df['target_label'] = df['labels'].apply(get_first_target_label)
df = df[df['target_label'].notnull()].reset_index(drop=True)

# Map emotion string to class index 0-6
df['label'] = df['target_label'].map(target_label2id)
df = df[['text', 'label']]  # Final columns: text + class index

df.head()

,text,label
0,To make her feel threatened,2
1,"Shit, I guess I accidentally bought a Pay-Per-...",6
2,"I never thought it was at the same moment, but...",4
3,i got a bump and a bald spot. i feel dumb <3,6
4,I miss them being alive,1


In [8]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=42)

In [9]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

# Convert pandas back to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


/Users/bennyxiong/Documents/Source/ML/CSCN8010_FinalProject/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 395/395 [00:00<00:00, 32082.69 examples/s]


In [ ]:
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch

# Model setup
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(target_emotions)
)

# Metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/Users/bennyxiong/Documents/Source/ML/CSCN8010_FinalProject/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- T

In [11]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/bennyxiong/Documents/Source/ML/CSCN8010_FinalProject/.venv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3555
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2230
  Number of trainable parameters = 109487623
  0%|          | 0/2230 [00:00<?, ?it/s]/Users

{'loss': 1.5661, 'learning_rate': 1.9103139013452916e-05, 'epoch': 0.45}


  9%|▉         | 200/2230 [05:06<51:32,  1.52s/it]

{'loss': 1.1999, 'learning_rate': 1.820627802690583e-05, 'epoch': 0.9}


 10%|█         | 223/2230 [05:41<39:53,  1.19s/it]The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 64
/Users/bennyxiong/Documents/Source/ML/CSCN8010_FinalProject/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
                                                  
 10%|█         | 223/2230 [05:51<39:53,  1.19s/it]Saving model checkpoint to ./results/checkpoint-223
Configuration saved in ./results/checkpoint-223/config.json


...

 70%|███████   | 1561/2230 [59:19<13:22,  1.20s/it]The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 64
                                                   
 70%|███████   | 1561/2230 [59:30<13:22,  1.20s/it]Saving model checkpoint to ./results/checkpoint-1561
Configuration saved in ./results/checkpoint-1561/config.json


{'eval_loss': 1.29880690574646, 'eval_accuracy': 0.6683544303797468, 'eval_f1': 0.6684059761407459, 'eval_precision': 0.6744655569619256, 'eval_recall': 0.6683544303797468, 'eval_runtime': 10.6507, 'eval_samples_per_second': 37.087, 'eval_steps_per_second': 0.657, 'epoch': 7.0}


Model weights saved in ./results/checkpoint-1561/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-1115 (score: 0.6951084704894018).
 70%|███████   | 1561/2230 [59:31<25:30,  2.29s/it]

{'train_runtime': 3571.6963, 'train_samples_per_second': 9.953, 'train_steps_per_second': 0.624, 'train_loss': 0.5132844254116765, 'epoch': 7.0}


TrainOutput(global_step=1561, training_loss=0.5132844254116765, metrics={'train_runtime': 3571.6963, 'train_samples_per_second': 9.953, 'train_steps_per_second': 0.624, 'train_loss': 0.5132844254116765, 'epoch': 7.0})

In [ ]:
trainer.save_model("../../models/emotionClassifier")
tokenizer.save_pretrained("../../models/emotionClassifier")

Saving model checkpoint to model
Configuration saved in model/config.json
Model weights saved in model/pytorch_model.bin
tokenizer config file saved in model/tokenizer_config.json
Special tokens file saved in model/special_tokens_map.json


('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')